# Deploying A Sentence-Embedding Model

## Contents

1. [Setup](#Setup)
2. [Deploy](#Deploy)
3. [Evaluate](#Evaluate)
4. [Remove](#Remove)

# Setup

- Load environmental variables
- Create Sagemaker Session
- Define S3 Bucket, path and model name
- Provide IAM role

In [3]:
%load_ext dotenv
import json
import os
import sagemaker
from sagemaker.predictor import RealTimePredictor, json_serializer, json_deserializer
from sagemaker.pytorch import PyTorchModel

%dotenv -v

sagemaker_session = sagemaker.Session()

bucket = os.getenv("S3_BUCKET")
endpoint = os.getenv("ENPOINT_NAME")
entry = os.getenv("ENTRY_POINT")                                    # Python file to execute
instance_type = os.getenv("INSTANCE_TYPE")
model_location = f"{bucket}{os.getenv('MODEL_PATH')}"               # path to model data
py_ver = os.getenv("PY_VER")
role = sagemaker.get_execution_role()                               # IAM role
source = os.getenv("SOURCE_DIR")                                    # Source for code
version = os.getenv("FRAMEWORK")                                    # Pytorch Version

# git_conf = {'repo': 'https://github.com/sam3ay/sentence-compare.git', 'branch': 'code'}
git_config = {'repo': os.getenv("REPO"), 'branch': os.getenv("BRANCH")}

# Deploy

Provide custom hosting functions:
- input_fn: Deserializes input data
- model_fn: Loads pytorch model
- predict_fn: run inference with input data
- output_fn: Serializes results of predict_fn
- predictor_cls: Creates the predictor with the endpoint name

In [4]:
class JSONPredictor(RealTimePredictor):
    """Serializes and deserializes json
    """
    def __init__(self, endpoint_name, sagemaker_session):
        super(JSONPredictor, self).__init__(endpoint_name, sagemaker_session, json_serializer, json_deserializer)

In [5]:
model = PyTorchModel(model_data=model_location,
                     role=role,
                     framework_version=version,
                     entry_point=entry,
                     source_dir=source,
                     predictor_cls=JSONPredictor,
                     git_config=git_config,
                     py_version=py_ver)

In [6]:
predictor = model.deploy(initial_instance_count=1,
                         instance_type=instance_type,
                         endpoint_name=endpoint)

---------------!

# Evaluate

Determine Textual Similarity

In [9]:
address_true = {
    "query": ["Speak to people in a formal way."],
    "corpus": [
        "the particulars of the place where someone lives or an organization is situated.",
        "a formal speech delivered to an audience.",
        "write the name and address of the intended recipient on (an envelope, letter, or package).",
        "speak to (a person or an assembly), typically in a formal way.",
    ],
}

package_false = {
    "query": ["No Idea."],
    "corpus": [
        "an object or group of objects wrapped in paper or packed in a box.",
        "a set of proposals or terms offered or agreed as a whole.",
        "put into a box or wrapping for sale or transport.",
        "present (someone or something) in an attractive or advantageous way.",
    ],
}

address_false = {
    "query": ["Where I walk to."],
    "corpus": [
        "the particulars of the place where someone lives or an organization is situated.",
        "a formal speech delivered to an audience.",
        "write the name and address of the intended recipient on (an envelope, letter, or package).",
        "speak to (a person or an assembly), typically in a formal way.",
    ],
}

address_response = predictor.predict(address_true)
package_response = predictor.predict(package_false)
address_response_2 = predictor.predict(address_false)
print(address_response)
print(address_response_2)
print(package_response)

{'predict': 1, 'sentence': 'a formal speech delivered to an audience.'}
{'predict': 0, 'sentence': 'write the name and address of the intended recipient on (an envelope, letter, or package).'}
{'predict': 0, 'sentence': 'a set of proposals or terms offered or agreed as a whole.'}


# Remove

Remove the endpoint to prevent additional charges

In [10]:
sagemaker_session.delete_endpoint(predictor.endpoint)